# Contexte 
Le Problème du Voyageur de Commerce (TSP) pose la question suivante : « Étant donnée une liste de
villes et les distances entre chaque paire de villes, quel est le plus court trajet possible qui visite chaque
ville et revient à la ville d’origine ? » Il s’agit d’un problème N P-difficile en optimisation combinatoire,
important en recherche opérationnelle et en informatique théorique.
Le problème a été formulé pour la première fois en 1930 et est l’un des problèmes les plus étudiés
en optimisation. Il est utilisé comme benchmark pour de nombreuses méthodes d’optimisation. Bien que
le problème soit computationnellement difficile, de nombreuses heuristiques et algorithmes exacts sont
connus, de sorte que certaines instances avec des dizaines de milliers de villes peuvent être résolues com-
plètement et même des problèmes avec des millions de villes peuvent être approximés à une fraction de
1%.
Formellement, soit G = (V,E,w) un graphe complet non orienté tel que |V| = n et w : E → R≥0
la fonction de poids. C’est-à-dire que G est le graphe Kn avec des poids sur ses arêtes. Dans ce modèle,
chaque sommet représente une ville et chaque arête pondérée reliant deux sommets représente la distance
entre les villes correspondantes.
Ainsi, étant donné un graphe complet pondéré non orienté, le TSP consiste à trouver un cycle ha-
miltonien (un cycle qui traverse chaque sommet exactement une fois) qui minimise la somme de ses
poids.

# Fonctions

In [8]:
def get_data(path : str) -> list:
    """
    get_data("Projet/17.in")
    """
    D = []
    with open(path, "r") as f:
        N  = int(f.readline())

        for _ in range(N):
            D.append(list(map(int, f.readline().split())))
    return D

# Question 1
Décrire (d’autres) situations réelles qui peuvent être modélisées comme un TSP.
- Un ensemble de mots est donné. Le but est de passer par chaque mot en éffectuant le moins d'action possible. Chaque mot peut se transoformer en un autre en :
    - changeant une lettre
    - ajoutant une lettre
    - enlevant une lettre 
- Un telescope doit observer un certain nombre de planetes. Le changement de possition de la lunette est couteux. Le but est de prévoir le chemin le plus court passant par toutes les planètes, afin de réduire les coûts.
- Un personnage sur Amongus doit effectuer toutes ses tâches. Etant donné la présence d'imposteurs dans le groupe, il doit effectuer ses tâches le plus vite possible

sser par chaque mot en éffectuant le moins d'action possible. Chaque mot peut se transoformer en un autre en :
    - changeant une lettre
    - ajoutant une lettre
    - enlevant une lettre 
- Un telescope doit observer un certain nombre de planetes. Le changement de possition de la lunette est couteux. Le but est de prévoir le chemin le plus court passant par toutes les planètes, afin de réduire les coûts.
- Un personnage sur Amongus doit effectuer toutes ses tâches. Etant donné la présence d'imposteurs dans le groupe, il doit effectuer ses tâches le plus vite possible

# Question 2
Développer et implémenter un algorithme exact pour résoudre le TSP, utilisant la technique du
Branch and Bound.

In [ ]:
from math import inf

class Solution :
    def __init__(self, vars, fixed_vars) :
        # x[i][j] = 0/1
        self.vars = [[0] for i in range()]
        self.fixed_vars = fixed_vars
        self.left = None
        self.right = None

    def calculateSol(self, data):
        result = 0
        for var in self.vars :
            result += 
        return result

    def fix_var(self,i,j,value) : 
        self.fixed_vars.append([i,j,value])

    def copy(self) -> Solution :
        var_copy = []
        for var in self.vars : 
            var_copy.append(var)
        return Solution(var_copy)
    
    def addLeft(self, solution):
        self.left = solution

    def addRight(self, solution):
        self.right = solution

    def verify_cycle_hamiltonien():
        # Vérifier qu'on peut aller de chaque node à chaque node en un nombre fini d'étape
        pass

class Arbre :
    def __init__(self) :
        self.first = None,

    def addFirst():
        pass

    def findBestLeaf() -> Solution :
        pass

In [ ]:
get_data("Projet/100.in")

BEST_SOL = inf

[[0,
  1693,
  2252,
  1104,
  2523,
  397,
  1447,
  425,
  1424,
  749,
  902,
  1547,
  2554,
  2378,
  1015,
  1259,
  1018,
  1167,
  814,
  1405,
  885,
  1330,
  882,
  1206,
  1194,
  1511,
  1572,
  416,
  2081,
  2145,
  887,
  790,
  2641,
  1804,
  1576,
  1173,
  2518,
  1225,
  2102,
  1358,
  2698,
  661,
  2251,
  1668,
  923,
  2115,
  429,
  2523,
  442,
  1707,
  1129,
  2474,
  1010,
  1394,
  1708,
  795,
  1360,
  718,
  1031,
  1152,
  1043,
  1260,
  288,
  1366,
  1250,
  1286,
  593,
  1618,
  1241,
  1387,
  2635,
  802,
  1531,
  942,
  628,
  2649,
  1048,
  2442,
  1008,
  863,
  1220,
  2186,
  1716,
  888,
  1734,
  1508,
  1255,
  1219,
  806,
  644,
  753,
  303,
  413,
  1359,
  2431,
  2377,
  815,
  783,
  1468,
  2643],
 [1693,
  0,
  1708,
  2403,
  1186,
  2057,
  1392,
  1620,
  1343,
  2442,
  2359,
  1606,
  970,
  1662,
  2446,
  2831,
  2455,
  2837,
  2310,
  1603,
  2502,
  2897,
  1921,
  2895,
  893,
  2671,
  1737,
  1363,
  1501,
  119

# QUESTION 3
Développer et implémenter une heuristique constructive pour résoudre le TSP.